In [127]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, POST

import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
sparql = SPARQLWrapper2("https://lod.lobbywatch.ch/query/")
sparql.setQuery("""
    PREFIX or: <http://openresearch.org/vocab/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>

    SELECT DISTINCT ?politician ?party ?job ?org_label ?role ?employmentType ?remuneration
    WHERE {

        ?mep_uri a dbo:MemberOfParliament;
               schema:memberOf ?party_uri;
               schema:familyName ?lastName;
               schema:givenName ?firstName.
        OPTIONAL {
        ?mep_uri schema:hasOccupation ?job_uri.
        ?job_uri <http://schema.org/name> ?job.	
        FILTER(LANG(?job) = "de")
      }

        ?party_uri a dbo:PoliticalParty;  
          schema:name ?party.

        OPTIONAL{
          ?compensation a lw:MembersInterestsCompensation;
              <http://www.w3.org/ns/org#memberOf> ?interests;
              <http://www.w3.org/ns/org#remuneration> ?remuneration.

          ?interests a <https://lod.lobbywatch.ch/MembersInterest>;
                   <https://lod.lobbywatch.ch/parliamentMember> ?mep_uri;
                  <http://www.w3.org/ns/org#organization> ?org;
                  <http://www.w3.org/ns/org#role> ?role;
                  <https://lod.lobbywatch.ch/employmentType> ?employmentType.

          ?org <http://schema.org/name> ?org_label.
          FILTER(LANG(?org_label) = "de")
        }

        BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .        
        FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician ?org_label
""")
sparql.setMethod(POST)
sparql.setReturnFormat(JSON)
results = sparql.query().bindings

In [56]:
df = pd.json_normalize(results)
for column in df.columns:
    df[column] = df[column].apply(lambda x: x.value if pd.notnull(x) else x)

df["remuneration"] = df["remuneration"].apply(lambda x: float(x))
df = df.groupby(["politician", "party", "job", "org_label", "employmentType", "remuneration"], dropna=False).agg(lambda x: set(x)).reset_index()

In [107]:
# List of politicians that did not disclose their income
disclosures = df[["politician", "party", "remuneration"]].groupby(["politician", "party"]).agg(lambda x: len(set(x).difference(set([1, float("nan")]))) != 0).reset_index().rename(columns={"remuneration": "has_disclosed_income"})
disclosures[~disclosures["has_disclosed_income"]].sort_values(by=["party"])

,politician,party,has_disclosed_income
207,Ruth Humbel,Christlichdemokratische Volkspartei,False
185,Philipp Kutter,Christlichdemokratische Volkspartei,False
175,Nicolo Paganini,Christlichdemokratische Volkspartei,False
128,Lilian Studer,Evangelische Volkspartei,False
3,Alex Farinelli,Freisinnig-Demokratische Partei,False
187,Philippe Bauer,Freisinnig-Demokratische Partei,False
106,Jacques Bourgeois,Freisinnig-Demokratische Partei,False
152,Martin Schmid,Freisinnig-Demokratische Partei,False
17,Anna Giacometti,Freisinnig-Demokratische Partei,False
105,Jacqueline de Quattro,Freisinnig-Demokratische Partei,False


In [139]:
shame_table = disclosures.drop(columns = "politician").groupby(["party"]).agg([np.mean, len, lambda x: len(x) - sum(x)]).reset_index()
shame_table.columns = ["party", "ratio_disclosed", "members", "nondisclosing_members"]
shame_table = shame_table.sort_values(by=["ratio_disclosed", "members"])

shame_table

,party,ratio_disclosed,members,nondisclosing_members
5,Evangelische Volkspartei,0.666667,3,1
12,Schweizerische Volkspartei,0.796610,59,12
6,Freisinnig-Demokratische Partei,0.850000,40,6
3,Christlichdemokratische Volkspartei,0.918919,37,3
8,Grünliberale Partei,0.937500,16,1
7,Grüne Partei der Schweiz,0.967742,31,1
0,Basels starke Alternative,1.000000,1,0
2,CVP Oberwallis,1.000000,1,0
4,Eidgenössisch-Demokratische Union,1.000000,1,0
9,Lega dei Ticinesi,1.000000,1,0


In [140]:
import plotly.express as px
fig = px.bar(shame_table, x="ratio_disclosed", y="party", hover_data=["members", "nondisclosing_members"], orientation='h')
fig.show()

In [16]:
# Organisations represented directly and indirectly
# Most represented
# Represented @ which party
sum(df["politician"] == "Marco Romano")

1

In [140]:
query = """
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?politician ?party ?org ?org_uri ?org_id
    WHERE {

      ?mep a dbo:MemberOfParliament;
          schema:memberOf ?party_uri;
          schema:familyName ?lastName;
          schema:givenName ?firstName.

      BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .

      ?party_uri a dbo:PoliticalParty;
                 skos:altLabel ?party.

      ?interests a lw:MembersInterest;
        org:organization ?org_uri;
        lw:parliamentMember ?mep.

      OPTIONAL{
        ?org_uri <http://www.w3.org/ns/org#identifier> ?org_id
      }
      ?org_uri schema:name ?org.

      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician
"""

sparql.setQuery(query)
results = sparql.query().bindings

direct_influence = pd.json_normalize(results)
for column in direct_influence.columns:
    direct_influence[column] = direct_influence[column].apply(lambda x: x.value if pd.notnull(x) else x)

In [141]:
query = """
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?politician ?party ?org ?org_uri ?org_id
    WHERE {

      ?mep a dbo:MemberOfParliament;
          schema:memberOf ?party_uri;
          schema:familyName ?lastName;
          schema:givenName ?firstName.

      BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .

      ?party_uri a dbo:PoliticalParty;
                 skos:altLabel ?party.

      ?access a lw:AccessRight;
          lw:issuedBy ?mep;
          lw:issuedTo ?person.
      ?person org:hasMembership ?membership .
      ?membership org:organization ?org_uri.
      ?org_uri schema:name ?org.

      OPTIONAL{
        ?org_uri <http://www.w3.org/ns/org#identifier> ?org_id.
      }

      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician
"""

sparql.setQuery(query)
results = sparql.query().bindings

indirect_influence = pd.json_normalize(results)
for column in indirect_influence.columns:
    indirect_influence[column] = indirect_influence[column].apply(lambda x: x.value if pd.notnull(x) else x)

In [153]:
all_influences = pd.concat([direct_influence, indirect_influence])

In [163]:
df = all_influences #all_influences, direct_influence, indirect_influence

orgs_influence = df[["org", "org_uri", "politician"]].groupby(["org", "org_uri"], dropna=False).count().reset_index().sort_values(by=["politician"], ascending=False)
orgs_influence.columns = ["org", "org_uri", "influenced_politicians"]
orgs_influence[0:20]

,org,org_uri,influenced_politicians
2625,Sport,https://lod.lobbywatch.ch/organisation/1751,140
3277,Wohn- und Grundeigentum,https://lod.lobbywatch.ch/organisation/2899,92
307,Biodiversität und Artenschutz,https://lod.lobbywatch.ch/organisation/637,90
297,"Bildung, Forschung und Innovation",https://lod.lobbywatch.ch/organisation/2124,89
349,Bundeshaus-Band,https://lod.lobbywatch.ch/organisation/1750,81
214,Auslandschweizer,https://lod.lobbywatch.ch/organisation/1278,79
1163,"Gruppa parlamentara ""lingua e cultura rumantscha""",https://lod.lobbywatch.ch/organisation/5964,78
3273,Wirtschafts- und währungspolitischer Arbeitskreis,https://lod.lobbywatch.ch/organisation/5978,65
2181,Rotes Kreuz,https://lod.lobbywatch.ch/organisation/5024,61
679,Erneuerbare Energien,https://lod.lobbywatch.ch/organisation/1505,58


In [156]:
# Number of members in each party:
parties = direct_influence[["politician", "party"]].drop_duplicates().groupby("party").count().reset_index().sort_values(by="politician", ascending = False)
parties[parties["politician"] > 5]

,party,politician
13,SVP,59
12,SP,48
6,FDP,40
2,CVP,37
8,GPS,31
7,GLP,16


In [157]:
relevant_parties = list(parties[parties["politician"] > 5]["party"])

In [213]:
indirect_influence[["politician","party", "org"]]

,politician,party,org
0,Adèle Thorens Goumaz,GPS,Greenpeace Schweiz
1,Adèle Thorens Goumaz,GPS,Agrarallianz
2,Adèle Thorens Goumaz,GPS,WWF Schweiz
3,Albert Rösti,SVP,Berner Bauern Verband Management AG
4,Albert Rösti,SVP,Kanton Bern
...,...,...,...
1548,Yvonne Feri,SP,IV-Allianz
1549,Yvonne Feri,SP,Schweizerischer Musikpädagogischer Verband
1550,Yvonne Feri,SP,Allianz Ergänzungsleistungen
1551,Yvonne Feri,SP,Inclusion Handicap


In [162]:
df = indirect_influence #all_influences, direct_influence, indirect_influence

influences = pd.pivot_table(df[["politician","party", "org"]], values='politician', index=['org'], columns=['party'], aggfunc=len).fillna(0)
influences["total"] = influences.sum(axis=1)

influences = influences[["total"] + relevant_parties]
influences[relevant_parties] = influences[relevant_parties].div(influences.total, axis = 0)
influences.sort_values(by="total", ascending = False)[0:10]

party,total,SVP,SP,FDP,CVP,GPS,GLP
org,,,,,,,
Schweizerische Public Affairs Gesellschaft,44.00,0.23,0.23,0.23,0.16,0.02,0.05
Schweizerischer Gewerkschaftsbund,13.00,0.00,0.69,0.00,0.00,0.31,0.00
FDP Die Liberalen Schweiz,8.00,0.12,0.00,0.75,0.12,0.00,0.00
WWF Schweiz,7.00,0.00,0.43,0.00,0.00,0.57,0.00
Schweizer Bauernverband,6.00,0.33,0.00,0.17,0.50,0.00,0.00
Schweizerischer Gewerbeverband,5.00,0.20,0.00,0.60,0.20,0.00,0.00
Kommission für Wirtschaftspolitik,5.00,0.00,0.80,0.00,0.20,0.00,0.00
Alliance Sud,5.00,0.00,0.60,0.00,0.00,0.40,0.00
Universität Zürich,5.00,0.20,0.40,0.00,0.00,0.20,0.20


In [243]:
query = """
    PREFIX http: <http://www.w3.org/2011/http#>
    PREFIX or: <http://openresearch.org/vocab/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>

    SELECT DISTINCT ?lobbyName ?sectorName ?org ?lobbyist ?party
    WHERE {
      ?lobby a lw:Lobby;
         rdfs:label ?lobbyName;
        lw:businessSector ?sector ;
        schema:member ?member .
      ?sector rdfs:label ?sectorName .

      ?member schema:familyName ?lastName;
              schema:givenName ?firstName.

      OPTIONAL{
        ?access lw:issuedTo ?member.
        ?access lw:issuedBy ?politician.
        ?politician schema:memberOf ?political_org.
        ?political_org a dbo:PoliticalParty.
        ?political_org <http://www.w3.org/2004/02/skos/core#altLabel> ?party.

        ?member schema:hasOccupation ?occupation_uri;
               org:hasMembership ?membership.
        ?membership <http://www.w3.org/ns/org#organization> ?org_uri.
        ?org_uri <http://schema.org/name> ?org.
      }
      BIND(CONCAT(?firstName, " ", ?lastName) AS ?lobbyist) .

      FILTER(LANG(?lobbyName) = "de")
      FILTER(LANG(?sectorName) = "de")
      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
"""

sparql.setQuery(query)
results = sparql.query().bindings

df = pd.json_normalize(results)
for column in df.columns:
    df[column] = df[column].apply(lambda x: x.value if pd.notnull(x) else x)

In [244]:
lobbies = dict()
for result in results:

    org = result["org"].value
    subsector = result["lobbyName"].value
    sector = result["sectorName"].value

    if sector in lobbies:
        
        if subsector in lobbies[sector]:
            lobbies[sector][subsector].add(org)
        else:
            lobbies[sector][subsector] = set([org])
        
    else:
        lobbies[sector] = {
            subsector: set([org]),
        }
    

In [245]:
# Companies having its stakes in lobby
lobbies["Gesundheit"]["Krankenkassen"]

{'ARA Kiesental-Aaretal ARAKA AG',
 'Agroscope',
 'BWZ Berufs- und Weiterbildungszentrum Lyss',
 'CSS Versicherung AG',
 'EAE (Energieagentur Elektrogeräte)',
 'Gemeinde Konolfingen',
 'Groupe Mutuel Services SA',
 'Pink Cross',
 'Santésuisse',
 'Schweizerische Volkspartei',
 'Schweizerische Volkspartei International',
 'Swiss Diversity Award',
 'digital.swiss'}

In [246]:
subset = df[["lobbyName", "lobbyist", "sectorName"]].drop_duplicates().groupby(["lobbyName", "sectorName"]).count().reset_index().sort_values(by="lobbyist", ascending=False)
subset[0:10]

,lobbyName,sectorName,lobbyist
16,Consulting/PR,Wirtschaft,40
4,Arbeitnehmerorganisationen,Wirtschaft,14
68,Umwelt & Natur,Umwelt,12
13,Bildung/Wissenschaft,Bildung,11
39,Kantone/Regionen,Staatspolitik/Staatswirtschaft,10
67,Tourismus/Gastronomie,Wirtschaft,8
2,Advokaturen/Treuhand,Wirtschaft,8
44,Landwirtschaft allgemein,Landwirtschaft,8
38,KMU/Gewerbe/Arbeitgeber,Wirtschaft,8
8,Banken,Wirtschaft,7


In [247]:
lobbies["Wirtschaft"]["Banken"]

{'Bank Julius Bär & Co. AG',
 'Bündner Offiziersgesellschaft',
 'CVP Baden',
 'FDP Die Liberalen Fällanden',
 'FDP Die Liberalen Graubünden',
 'FDP Die Liberalen Schweiz',
 'FDP die Liberalen Baar',
 'Freidenker-Vereinigung der Schweiz',
 'Gemeinde Fällanden',
 'HarbourClub, Chief Communications Officers',
 'Jungfreisinnige Graubünden',
 'LZ Medien Holding AG',
 'Raiffeisen Schweiz',
 'SAMDalumni',
 'SIX Group AG',
 'SOG Schweizerische Offiziersgesellschaft',
 'Schweizerische Bankiervereinigung',
 'Schweizerische Offiziersgesellschaft der Artillerie',
 'Stadt Baden',
 'Surseer Woche AG',
 'Universität Luzern',
 'Verband Schweizerischer Kantonalbanken',
 'Vereinigung Schweizerischer Assetmanagement- und Vermögensverwaltungsbanken VAV',
 'Vereinigung Schweizerischer Privatbanken'}

In [248]:
parties = df[["lobbyName", "sectorName", "lobbyist", "party"]].drop_duplicates()
lobbysts = pd.pivot_table(parties, values='lobbyist', index=['lobbyName', "sectorName"], columns=['party'], aggfunc=len).fillna(0)
lobbysts["total"] = lobbysts.sum(axis=1)

lobbysts = lobbysts[["total"] + relevant_parties]
lobbysts.sort_values(by="total", ascending = False)[0:10]

,party,total,SVP,SP,FDP,CVP,GPS,GLP
lobbyName,sectorName,,,,,,,
Consulting/PR,Wirtschaft,40.00,10.00,11.00,8.00,5.00,1.00,3.00
Arbeitnehmerorganisationen,Wirtschaft,14.00,0.00,10.00,0.00,0.00,4.00,0.00
Umwelt & Natur,Umwelt,12.00,0.00,2.00,0.00,0.00,9.00,0.00
Bildung/Wissenschaft,Bildung,11.00,1.00,6.00,2.00,2.00,0.00,0.00
Kantone/Regionen,Staatspolitik/Staatswirtschaft,10.00,1.00,2.00,2.00,2.00,2.00,0.00
Tourismus/Gastronomie,Wirtschaft,8.00,2.00,0.00,0.00,5.00,1.00,0.00
Advokaturen/Treuhand,Wirtschaft,8.00,3.00,0.00,3.00,2.00,0.00,0.00
Landwirtschaft allgemein,Landwirtschaft,8.00,4.00,0.00,1.00,3.00,0.00,0.00
KMU/Gewerbe/Arbeitgeber,Wirtschaft,8.00,2.00,0.00,4.00,2.00,0.00,0.00
